# Aula 07 - Filtragem Baseada em Conhecimento - Exercícios

## Importação dos dados (MovieLens 100k)

In [1]:
import pandas as pd
import numpy as np

In [2]:
import wget
!python3 -m wget https://files.grouplens.org/datasets/movielens/ml-100k.zip
# !tar -xvzf ml-100k.zip
!unzip -o ml-100k.zip


100% [......................................................] 4924029 / 4924029
Saved under ml-100k.zip
Archive:  ml-100k.zip
   creating: ml-100k/
  inflating: ml-100k/allbut.pl       
  inflating: ml-100k/mku.sh          
  inflating: ml-100k/README          
  inflating: ml-100k/u.data          
  inflating: ml-100k/u.genre         
  inflating: ml-100k/u.info          
  inflating: ml-100k/u.item          
  inflating: ml-100k/u.occupation    
  inflating: ml-100k/u.user          
  inflating: ml-100k/u1.base         
  inflating: ml-100k/u1.test         
  inflating: ml-100k/u2.base         
  inflating: ml-100k/u2.test         
  inflating: ml-100k/u3.base         
  inflating: ml-100k/u3.test         
  inflating: ml-100k/u4.base         
  inflating: ml-100k/u4.test         
  inflating: ml-100k/u5.base         
  inflating: ml-100k/u5.test         
  inflating: ml-100k/ua.base         
  inflating: ml-100k/ua.test         
  inflating: ml-100k/ub.base         
  inflating: ml-

In [3]:
#Types of genres
genre = pd.read_csv('./ml-100k/u.genre', sep="|", encoding='latin-1', header=None)
genre.drop(genre.columns[1], axis=1, inplace=True)
genre.columns = ['Genres']
genre_list = list(genre['Genres'])
genre_list

['unknown',
 'Action',
 'Adventure',
 'Animation',
 "Children's",
 'Comedy',
 'Crime',
 'Documentary',
 'Drama',
 'Fantasy',
 'Film-Noir',
 'Horror',
 'Musical',
 'Mystery',
 'Romance',
 'Sci-Fi',
 'Thriller',
 'War',
 'Western']

In [4]:
#Types of occupations
occupation = pd.read_csv('./ml-100k/u.occupation', sep="|", encoding='latin-1', header=None)
occupation.columns = ['Occupations']
occupation_list = list(occupation['Occupations'])
occupation_list

['administrator',
 'artist',
 'doctor',
 'educator',
 'engineer',
 'entertainment',
 'executive',
 'healthcare',
 'homemaker',
 'lawyer',
 'librarian',
 'marketing',
 'none',
 'other',
 'programmer',
 'retired',
 'salesman',
 'scientist',
 'student',
 'technician',
 'writer']

In [5]:
#Load the Ratings data
data = pd.read_csv('./ml-100k/u.data', sep="\t", header=None)
data.columns = ['userId', 'movieId', 'rating', 'timestamp']
data.head()

,userId,movieId,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [6]:
#Load the Movies data
item = pd.read_csv('./ml-100k/u.item', sep="|", encoding='latin-1', header=None)
item.columns = ['movieId', 'title' ,'release','video release date', 'IMDb URL', 'unknown', 'Action', 
                'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 
                'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
item['release'] = pd.to_datetime(item['release'])
item = item[pd.notnull(item['release'])]
item['year'] = item['release'].dt.year.astype(int)
item.drop(columns=['release', 'video release date', 'IMDb URL'], inplace=True)
item.head()

,movieId,title,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,...,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western,year
0,1,Toy Story (1995),0,0,0,1,1,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
1,2,GoldenEye (1995),0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
2,3,Four Rooms (1995),0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1995
3,4,Get Shorty (1995),0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1995
4,5,Copycat (1995),0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,1995


In [7]:
df_meta = item.melt(id_vars=['movieId', 'title'], var_name='genre')
df_meta = df_meta[df_meta.value == 1]
df_meta.drop(columns=['value'], inplace=True)
# df_meta[df_meta['movieId']==1]
df_meta.head()

,movieId,title,genre
1371,1373,Good Morning (1971),unknown
1682,2,GoldenEye (1995),Action
1684,4,Get Shorty (1995),Action
1697,17,From Dusk Till Dawn (1996),Action
1701,21,Muppet Treasure Island (1996),Action


In [8]:
# Obter a lista de gêneros de um item
def get_genres(df, movieId):
    if movieId not in df['movieId'].values:
        return []
    return df.loc[(df.movieId==movieId),'genre'].tolist()

get_genres(df_meta, 1)

['Animation', "Children's", 'Comedy']

In [9]:
#Load the User data
user = pd.read_csv('./ml-100k/u.user', sep="|", encoding='latin-1', header=None)
user.columns = ['userId', 'age', 'gender', 'occupation', 'zip code']
user.head()

,userId,age,gender,occupation,zip code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213


***Exercício 01:*** Implemente um sistema de recomendação baseado em casos que permite as seguintes restrições definidas pelo usuário:
- Escolher entre filmes mais antigos ou mais recentes
- Determinar os gêneors de maior preferência (mas não restrito a eles)
- Escolher os filmes mais populares ou os menos populares dentre os usuários de uma dada profissão.
- Atribuir importância maior ou menor para cada uma das restrições anteriores.

Recomende uma lista de 10 filmes com base nos critérios definidos pelo usuário.

Exemplo: usuário define:
- filmes_mais_recentes = True
- generos_preferidos = ['Drama']
- filmes_mais_populares = True
- profissão = 'writer'
- pesos = [0.33, 0.33, 0.33]

In [10]:
def jaccard_similarity(set1, set2):
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

In [11]:
# Occupation Preference DataFrame
op = (data
    # Merge with user data
    .merge(user, on='userId', how='inner')[['movieId', 'rating', 'occupation']]
    
    # Group by occupation and movieId and calculate the mean rating
    .groupby(['occupation', 'movieId']).mean()
    
    # Reset the index so we have a regular table
    .reset_index()
)

op.head()

,occupation,movieId,rating
0,administrator,1,4.000
1,administrator,2,3.600
2,administrator,3,3.000
3,administrator,4,3.375
4,administrator,5,3.000


In [12]:
# Get min rating for each occupation
op_min = op.groupby('occupation')['rating'].min().reset_index()

# Get max rating for each occupation
op_max = op.groupby('occupation')['rating'].max().reset_index()

op_min_max = op_min.merge(op_max, on='occupation', suffixes=('_min', '_max'))
op_min_max['diff'] = op_min_max['rating_max'] - op_min_max['rating_min']

op_min_max.head()

,occupation,rating_min,rating_max,diff
0,administrator,1.0,5.0,4.0
1,artist,1.0,5.0,4.0
2,doctor,1.0,5.0,4.0
3,educator,1.0,5.0,4.0
4,engineer,1.0,5.0,4.0


In [13]:
# Merge `op` with `op_min_max` to get the min and max rating for each occupation
occ_scores = op.merge(op_min_max, on='occupation')

# Calculate w_max and w_min for each movie within each occupation
occ_scores['w_max'] = (occ_scores['rating'] - occ_scores['rating_min']) / occ_scores['diff']
occ_scores['w_min'] = (occ_scores['rating_max'] - occ_scores['rating']) / occ_scores['diff']

occ_scores = occ_scores[['occupation', 'movieId', 'w_max', 'w_min']]

occ_scores.head()

,occupation,movieId,w_max,w_min
0,administrator,1,0.75000,0.25000
1,administrator,2,0.65000,0.35000
2,administrator,3,0.50000,0.50000
3,administrator,4,0.59375,0.40625
4,administrator,5,0.50000,0.50000


In [14]:
recency = item.copy()[['movieId', 'year']]

min_year = recency['year'].min()
max_year = recency['year'].max()
delta_year = max_year - min_year

recency['w_max'] = (recency['year'] - min_year) / delta_year
recency['w_min'] = (max_year - recency['year']) / delta_year

recency.head()

,movieId,year,w_max,w_min
0,1,1995,0.960526,0.039474
1,2,1995,0.960526,0.039474
2,3,1995,0.960526,0.039474
3,4,1995,0.960526,0.039474
4,5,1995,0.960526,0.039474


In [15]:
# Pre-Generating the Genre DataFrame 
genre_sets = df_meta.copy().groupby('movieId')['genre'].apply(set).reset_index()

genre_sets.head()

,movieId,genre
0,1,"{Children's, Animation, Comedy}"
1,2,"{Thriller, Adventure, Action}"
2,3,{Thriller}
3,4,"{Comedy, Drama, Action}"
4,5,"{Crime, Thriller, Drama}"


In [16]:
most_recent_movies = True
prefered_genres = ['Drama']
most_popular_movies = True
profession = 'writer'
weights = [0.33, 0.33, 0.33]

def recomend(
    most_popular_movies : bool = True, # Should we go for the most popular or least popular movies for a given occupation?
    prefered_genres : list[str] | None = None, # List of genres that the user likes
    profession : str | None = None, # The user's profession
    weights : list[float | int] | None = None,
    most_recent_movies : bool = False,
    n : int = 10,
) -> pd.DataFrame:
    if weights is None:
        weights = [1/len(prefered_genres) for _ in prefered_genres]
    
    # Normalizing the weights
    weights = np.array(weights) / np.sum(weights)
    
    # Score for recency
    r_recency = recency.copy()
    r_recency.rename(columns={'w_max' if most_recent_movies else 'w_min': 'w_recency'}, inplace=True)
    r_recency = r_recency[['movieId', 'w_recency']]
    
    # Score for genre
    r_genre = genre_sets.copy()
    genres_of_interest = set(prefered_genres)
    r_genre['w_genre'] = r_genre['genre'].apply(lambda x: jaccard_similarity(x, genres_of_interest))
    
    # Score for occupation  
    r_occupation = occ_scores.copy()[occ_scores['occupation'] == profession]
    r_occupation.rename(columns={'w_max' if most_popular_movies else 'w_min': 'w_occ'}, inplace=True)
    r_occupation = r_occupation[['movieId', 'w_occ']]
    
    # Overall score
    # Outer join to preserve all movies
    r = (r_recency
        .merge(r_genre, on='movieId', how='outer')
        .merge(r_occupation, on='movieId', how='outer')
    )
    
    # Presume score is 0 for missing values
    r = r.fillna(0)
    
    # Accumulate the score using the weights
    r['score'] = (
          weights[0] * r['w_recency']
        + weights[1] * r['w_genre']
        + weights[2] * r['w_occ']
    )
    
    # Merge with item to get the title
    recomendation = r.merge(item, on='movieId', how='inner')[['movieId', 'title', 'score']]
    recomendation.sort_values('score', ascending=False, inplace=True)
    
    # Recommend the top 10 movies
    return recomendation.head(n)

In [17]:
# Example
recomend(
    most_popular_movies = True,
    prefered_genres = ['Drama'],
    profession = 'writer',
    weights = [0.01, 0.33, 0.03],
    most_recent_movies = True
)

,movieId,title,score
1386,1388,Gabbeh (1996),0.999644
1156,1158,"Fille seule, La (A Single Girl) (1995)",0.999289
1522,1524,Kaspar Hauser (1993),0.999289
1534,1536,Aiqing wansui (1994),0.999289
17,18,"White Balloon, The (1995)",0.998933
1111,1113,Mrs. Parker and the Vicious Circle (1994),0.998578
898,900,Kundun (1997),0.992888
849,851,Two or Three Things I Know About Her (1966),0.988620
191,192,Raging Bull (1980),0.988531
295,297,Ulee's Gold (1997),0.986131


***Exercício 02:*** Considere um CSP definido como uma tripla (V, D, C) onde:
- V = V_usuario U V_filme : variáveis relacionadas com o usuário e o filme, respectivamente
- C = C_usuario U C_filme U C_ui: restrições do usuário, filme e compatibilidade usuário/filme
- D = conjunto de domínios finitos para as variáveis

Podemos definir as variáveis e restrições como:
- V_usuario : {companhia(sozinho, casal, familia), filme_popular(sim, nao), filme_classico(sim, nao)}
- V_filme : {ano_lancamento(int), generos(list), n_avaliacoes(int)}
- C_usuario : {companhia=casal -> filme_popular=sim ^ companhia=sozinho -> filme_classico=nao}
- C_filme : {lista de itens do catalogo}
- C_ui : {companhia=casal -> generos=[Romance, Drama, Comedy], companhia=familia -> generos=[Children's], filme_classico=sim -> ano_lancamento < 1990, filme_classico=nao -> ano_lancamento > 1993, filme_popular=sim -> n_avaliacoes > 100, companhia=sozinho -> generos=[Horror, War]}

Dada a requisição: REQ={companhia=sozinho, filme_popular=sim}, recomende uma lista de 10 filmes para o usuário.

In [18]:
from copy import deepcopy

In [19]:
def expand_constraints(constraints : dict):
    constraints = deepcopy(constraints) # Avoid modifying the original
    
    match constraints['company']:
        case 'couple':
            constraints['popular'] = True
            constraints['genres'] = ['Romance', 'Drama', 'Comedy']
        case 'family':
            constraints['genres'] = ['Children\'s']
        case 'alone':
            constraints['classic'] = False
            constraints['genres'] = ['Horror', 'War']
    
    if constraints['classic']:
        constraints['year_upper_bound'] = 1990
    else:
        constraints['year_lower_bound'] = 1993
        
    if constraints['popular']:
        constraints['num_ratings_lower_bound'] = 100
        
    return constraints

In [20]:
num_ratings = data.groupby('movieId').size().reset_index(name='num_ratings')

pre_filter = (df_meta
    .merge(item[['movieId', 'year']], on='movieId')
    .merge(num_ratings[['movieId', 'num_ratings']], on='movieId')
)

pre_filter.head()

,movieId,title,genre,year,num_ratings
0,1373,Good Morning (1971),unknown,1971,1
1,2,GoldenEye (1995),Action,1995,131
2,4,Get Shorty (1995),Action,1995,209
3,17,From Dusk Till Dawn (1996),Action,1996,92
4,21,Muppet Treasure Island (1996),Action,1996,84


In [21]:
def apply_constraits(
    constraints : dict, 
    strategy : str | None = "most-popular", 
    n : int = 10,
) -> pd.DataFrame:
    
    filtered_df = pre_filter.copy()
    
    if 'genres' in constraints:
        filtered_df = filtered_df[filtered_df['genre'].isin(constraints['genres'])]
        
    if 'year_lower_bound' in constraints:
        filtered_df = filtered_df[filtered_df['year'] > constraints['year_lower_bound']]
    
    if 'year_upper_bound' in constraints:
        filtered_df = filtered_df[filtered_df['year'] < constraints['year_upper_bound']]
        
    if 'num_ratings_lower_bound' in constraints:
        filtered_df = filtered_df[filtered_df['num_ratings'] > constraints['num_ratings_lower_bound']]
        
    if 'num_ratings_upper_bound' in constraints:
        filtered_df = filtered_df[filtered_df['num_ratings'] < constraints['num_ratings_upper_bound']]
    
    match strategy:
        case "most-popular":
            filtered_df.sort_values('num_ratings', ascending=False, inplace=True)
        case "least-popular":
            filtered_df.sort_values('num_ratings', ascending=True, inplace=True)
        case "most-recent":
            filtered_df.sort_values('year', ascending=False, inplace=True)
        case "least-recent":
            filtered_df.sort_values('year', ascending=True, inplace=True)
        case "random":
            filtered_df = filtered_df.sample(frac=1)
        case _:
            pass
        
    return filtered_df[['movieId', 'title']].head(n)

In [22]:
# Setting the constraits
constraints = expand_constraints({
    'company': 'alone',
    'popular': True,
})

constraints

{'company': 'alone',
 'popular': True,
 'classic': False,
 'genres': ['Horror', 'War'],
 'year_lower_bound': 1993,
 'num_ratings_lower_bound': 100}

In [23]:
# Applying the constraints
apply_constraits(constraints)

,movieId,title
2808,181,Return of the Jedi (1983)
2819,286,"English Patient, The (1996)"
1999,288,Scream (1996)
2802,121,Independence Day (ID4) (1996)
2799,69,Forrest Gump (1994)
2795,22,Braveheart (1995)
2817,245,"Devil's Own, The (1997)"
2824,471,Courage Under Fire (1996)
2815,235,Mars Attacks! (1996)
2818,271,Starship Troopers (1997)
